KNN 붓꽃의 품종 분류하기 모델

### **미해결**: batch 학습 시키고 싶으면 dataload어떻게 해야하지???

In [32]:
from sklearn.datasets import load_iris
#import
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy


In [3]:
# for reproducibility
random.seed(777)
torch.manual_seed(777)


In [21]:
# hyperparameters
training_epochs = 15
batch_size = 10

In [34]:
#load iris data
#dataset 로드하기
iris_dataset = load_iris()
# print(iris_dataset)
# #dictionary 형태.
dataset = pd.DataFrame(data=numpy.c_[iris_dataset['data'], iris_dataset['target']], columns=iris_dataset['feature_names']+['target']).astype('float32')

dataset.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [7]:
print(iris_dataset.keys())

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


In [18]:
print(iris_dataset['target'])
print(iris_dataset['feature_names'])
print(iris_dataset['target_names']) #['setosa' 'versicolor' 'virginica'] -> 3개의 labels
print(iris_dataset['data'].shape) #(150, 4) -> #samples = 150, #features = 4

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']
(150, 4)


One-hot Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelEncoder
# label_ids=label_encoder.fit_transform(labels) -> 여기선 이미 숫자로 나와서 안해도 돔.

onehot_encoder=OneHotEncoder(sparse=False)
label_ids = iris_dataset['target']
print(label_ids)
reshaped=label_ids.reshape(len(label_ids), 1)
onehot=onehot_encoder.fit_transform(reshaped)
print(onehot)

**Train dataset, Test dataset**

150개의 붓꽃 데이터를 두 그룹으로 나눈다.
scikit-learn은 데이터셋을 섞어서 나눠주는 train_test_split함수를 제공한다.

이함수는 전체 행중 75%를 훈련세트로 뽑고 나머지 25%는 테스트세트가 된다.(test_size 지정안해주면 default임)


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(iris_dataset['data'],iris_dataset['target'],test_size = 0.25,random_state=0)


print("shape of X_train is {}".format(X_train.shape))
print("shape of y_train is {}".format(y_train.shape))
print("shape of X_test is {}".format(X_test.shape))
print("shape of y_test is {}".format(y_test.shape))

shape of X_train is (112, 4)
shape of y_train is (112,)
shape of X_test is (38, 4)
shape of y_test is (38,)


In [40]:
reshaped=y_train.reshape(len(y_train), 1)
one_hot_ytrain=onehot_encoder.fit_transform(reshaped)
reshaped=y_test.reshape(len(y_test), 1)
one_hot_ytest = onehot_encoder.fit_transform(reshaped)
print(one_hot_ytrain)
print(one_hot_ytest) 

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0.

In [41]:
print(one_hot_ytrain.shape)
print(one_hot_ytest.shape)

(112, 3)
(38, 3)


In [23]:
#iris_traindataset = (X_train, y_train)

In [27]:
# dataset loader -> for 배치학습
#How???
data_loader = DataLoader(dataset=iris_dataset,
                                          batch_size=batch_size, # 배치 크기는 10
                                          shuffle=True,
                                          drop_last=True)

In [28]:
print(len(data_loader))

0


모델 구현 및 학습

In [35]:
linear = nn.Linear(4, 3, bias=True)

In [36]:
# 비용 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss() # 내부적으로 소프트맥스 함수를 포함하고 있음.
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [58]:
for epoch in range(1000): # 앞서 training_epochs의 값은 15로 지정함.
    
    optimizer.zero_grad()
    hypothesis = linear(torch.FloatTensor(X_train))
    cost = criterion(hypothesis, torch.FloatTensor(one_hot_ytrain))
    cost.backward()
    optimizer.step()
    if epoch % 100 == 0:
      print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(cost))

print('Learning finished')

Epoch: 0001 cost = 0.391706884
Epoch: 0101 cost = 0.243964568
Epoch: 0201 cost = 0.201429516
Epoch: 0301 cost = 0.174314499
Epoch: 0401 cost = 0.155463576
Epoch: 0501 cost = 0.141553998
Epoch: 0601 cost = 0.130834743
Epoch: 0701 cost = 0.122296855
Epoch: 0801 cost = 0.115318403
Epoch: 0901 cost = 0.109494902
Learning finished


테스트 데이터를 사용하여 모델을 테스트한다.

In [59]:
with torch.no_grad(): # torch.no_grad()를 하면 gradient 계산을 수행하지 않는다.
    torchX_test = torch.FloatTensor(X_test)
    torchY_test = torch.FloatTensor(one_hot_ytest)

    prediction = linear(torchX_test)
    print(prediction)
    print(torch.argmax(prediction, dim = 1))
    #print("y_test is {}".format(y_test))
    correct_prediction = torch.argmax(prediction, dim = 1) == torch.FloatTensor(y_test)
    #tensor형식이 아니면 오류난다!!!
    
    print(correct_prediction)
    #torch.argmax(prediction, dim = 1): 열을 없애는것. 각 행에서 가장 큰 값을 갖는 index값만 남음.
    #(38,1)형식의 벡터가 남고, 가장 높은 확률은 갖는 index만 나옴. -> 예측된 숫자값
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    #테스트 데이터에서 무작위로 하나를 뽑아서 예측을 해본다
    r = random.randint(0, len(X_test) - 1)
    X_single_data = X_test[r:r + 1]
    Y_single_data = y_test[r:r + 1]

    print('Label: ', Y_single_data.item())
    single_prediction = linear(torch.FloatTensor(X_single_data))
    prediction = torch.argmax(single_prediction, 1).item()
    #print(prediction)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())


tensor([[ -5.9515,   1.2997,   6.3176],
        [ -1.9362,   3.5528,   0.1151],
        [ 10.9099,   4.2282, -13.1071],
        [ -6.8569,   3.0423,   5.8624],
        [  8.2754,   3.9319, -10.4349],
        [ -7.0481,   1.2837,   7.5854],
        [  8.9396,   3.8454, -10.9885],
        [ -1.9146,   3.2150,   0.7043],
        [ -2.6591,   3.4703,   1.1771],
        [ -0.8805,   3.1512,  -0.4335],
        [ -6.1110,   2.6526,   5.1690],
        [ -1.3945,   2.9655,   0.3730],
        [ -2.7553,   3.0784,   1.4729],
        [ -2.5177,   3.1293,   1.3009],
        [ -2.8144,   2.8220,   1.7987],
        [  9.0789,   3.9330, -11.2335],
        [ -2.4797,   2.6670,   1.6009],
        [ -2.9350,   2.6503,   1.8990],
        [  7.2935,   3.7547,  -9.3832],
        [ 10.1116,   4.0036, -12.1508],
        [ -4.9837,   1.6317,   4.9789],
        [ -2.6336,   2.2706,   2.0495],
        [  6.9272,   3.6381,  -8.8751],
        [  6.8199,   3.5232,  -8.8053],
        [ -3.8230,   2.4485,   3.1835],
